In [1]:
import numpy as np
import pandas as pd
import plotly
import plotly.figure_factory as ff
import plotly.graph_objs as go
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [2]:
data = pd.read_csv('task_b.csv')
data=data.iloc[:,1:]

In [3]:
data.head()

,f1,f2,f3,y
0,-195.871045,-14843.084171,5.532140,1.0
1,-1217.183964,-4068.124621,4.416082,1.0
2,9.138451,4413.412028,0.425317,0.0
3,363.824242,15474.760647,1.094119,0.0
4,-768.812047,-7963.932192,1.870536,0.0


In [4]:
data.corr()['y']

f1    0.067172
f2   -0.017944
f3    0.839060
y     1.000000
Name: y, dtype: float64

In [5]:
data.std()

f1      488.195035
f2    10403.417325
f3        2.926662
y         0.501255
dtype: float64

In [6]:
X=data[['f1','f2','f3']].values
Y=data['y'].values
print(X.shape)
print(Y.shape)

(200, 3)
(200,)


# What if our features are with different variance 

<pre>
* <b>As part of this task you will observe how linear models work in case of data having feautres with different variance</b>
* <b>from the output of the above cells you can observe that var(F2)>>var(F1)>>Var(F3)</b>

> <b>Task1</b>:
    1. Apply Logistic regression(SGDClassifier with logloss) on 'data' and check the feature importance
    2. Apply SVM(SGDClassifier with hinge) on 'data' and check the feature importance

> <b>Task2</b>:
    1. Apply Logistic regression(SGDClassifier with logloss) on 'data' after standardization 
       i.e standardization(data, column wise): (column-mean(column))/std(column) and check the feature importance
    2. Apply SVM(SGDClassifier with hinge) on 'data' after standardization 
       i.e standardization(data, column wise): (column-mean(column))/std(column) and check the feature importance

</pre>

<h3><font color='blue'> Make sure you write the observations for each task, why a particular feautre got more importance than others</font></h3>

**Task1 - Feature importance using Logistic Regression**

In [7]:
clf_lr = linear_model.SGDClassifier(loss='log',penalty='l2',alpha=0.0001,eta0=0.0001,random_state=15,verbose=2,learning_rate="constant")
clf_lr

SGDClassifier(eta0=0.0001, learning_rate='constant', loss='log',
              random_state=15, verbose=2)

In [8]:
clf_lr.fit(X,Y)

-- Epoch 1
Norm: 1.08, NNZs: 3, Bias: -0.001751, T: 200, Avg. loss: 2516.147588
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 0.61, NNZs: 3, Bias: -0.001551, T: 400, Avg. loss: 2621.694380
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 0.35, NNZs: 3, Bias: -0.001850, T: 600, Avg. loss: 3285.222158
Total training time: 0.00 seconds.
-- Epoch 4
Norm: 0.64, NNZs: 3, Bias: -0.003527, T: 800, Avg. loss: 3142.216822
Total training time: 0.00 seconds.
-- Epoch 5
Norm: 0.48, NNZs: 3, Bias: -0.004027, T: 1000, Avg. loss: 3009.886714
Total training time: 0.00 seconds.
-- Epoch 6
Norm: 1.40, NNZs: 3, Bias: -0.003523, T: 1200, Avg. loss: 3032.001946
Total training time: 0.00 seconds.
Convergence after 6 epochs took 0.00 seconds


SGDClassifier(eta0=0.0001, learning_rate='constant', loss='log',
              random_state=15, verbose=2)

In [9]:
clf_lr.coef_,clf_lr.intercept_

(array([[ 0.37170471, -1.34463853,  0.12669033]]), array([-0.00352309]))

**Observations:**<br>
*In a Over all view, while computing W_T*xi, W1 will play major role in deciding the output as its having highest weight and if xi1 is negative then it will tend to be more positive class and if xi1 is positive then product will be more negative leading to a negative class*<br>
*1.As in Weights array, W1 is negative it will increase the probability of being output be negative(class 0) if xi is positive*<br>
*2.Remaining two weights W0 and W2 will increase the probability of being output be positive(class 1) if xi is positive, Here as W0 is more dominant , it will decide the output class to be 1 or 0 to the maximum extent*<br>
*3.Alternatively, W1 is negative it will increase the probability of being output be positive(class 0) if xi is negative*<br>
*4.Similarly, Remaining two weights W0 and W2 will increase the probability of being output be negative(class 0) if xi is negative, Here as W0 is more dominant , it will decide the output class to be 1 or 0 to the maximum extent*<br>

**Proof:**

In [10]:
w = clf_lr.coef_
y_tilda = []
for i in range(len(data)):
    temp = np.dot(w[0],np.array(data.iloc[i,0:3]))
    if(temp<0):
        y_tilda.append(0)
    elif(temp>0):
        y_tilda.append(1)
df = data.copy()
df["y_tilda"]=y_tilda
df[df["y_tilda"]!=df["y"]]
# so here we ended up with wrong output for 95 values with the optimised weight vector
# Also this is due to non-standardization of feature data and we are wrongly predicting
## nearly 50% of the data

,f1,f2,f3,y,y_tilda
4,-768.812047,-7963.932192,1.870536,0.0,1
5,192.093461,-12677.139687,3.456229,0.0,1
8,-31.315888,-15289.241646,3.882981,0.0,1
10,355.555296,-16420.510738,1.968104,0.0,1
11,-371.562551,-2334.218517,2.231601,0.0,1
...,...,...,...,...,...
191,-157.066935,12149.075908,6.944223,1.0,0
192,841.946716,-3756.578733,0.430296,0.0,1
195,119.423142,-2985.720392,0.929967,0.0,1
198,443.199825,-1053.252455,5.467800,0.0,1


**Task1 - Feature importance using SVM(hinge loss)**

In [11]:
clf_svm = linear_model.SGDClassifier(loss='hinge',penalty='l2',alpha=0.0001,eta0=0.0001,random_state=15,verbose=2,learning_rate="constant")
clf_svm

SGDClassifier(eta0=0.0001, learning_rate='constant', random_state=15, verbose=2)

In [12]:
clf_svm.fit(X,Y)

-- Epoch 1
Norm: 0.61, NNZs: 3, Bias: -0.001600, T: 200, Avg. loss: 2634.084615
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 0.68, NNZs: 3, Bias: -0.001100, T: 400, Avg. loss: 2593.136418
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 0.76, NNZs: 3, Bias: -0.000900, T: 600, Avg. loss: 3308.216351
Total training time: 0.00 seconds.
-- Epoch 4
Norm: 0.77, NNZs: 3, Bias: -0.002700, T: 800, Avg. loss: 3155.085896
Total training time: 0.00 seconds.
-- Epoch 5
Norm: 0.93, NNZs: 3, Bias: -0.002800, T: 1000, Avg. loss: 3080.501847
Total training time: 0.00 seconds.
-- Epoch 6
Norm: 0.43, NNZs: 3, Bias: -0.002700, T: 1200, Avg. loss: 3011.887174
Total training time: 0.00 seconds.
-- Epoch 7
Norm: 0.69, NNZs: 3, Bias: -0.002200, T: 1400, Avg. loss: 3002.132514
Total training time: 0.00 seconds.
Convergence after 7 epochs took 0.00 seconds


SGDClassifier(eta0=0.0001, learning_rate='constant', random_state=15, verbose=2)

In [13]:
clf_svm.coef_,clf_svm.intercept_

(array([[ 0.38249139, -0.55764501,  0.15407861]]), array([-0.0022]))

**Observations:**<br>
*In a Over all view, while computing W_T*xi, W1 will play major role in deciding the output as its having highest weight and if xi1 is negative then it will tend to be more positive class and if xi1 is positive then product will be more negative leading to a negative class*<br>
*1.As in Weights array, W1 is negative it will increase the probability of being output be negative(class 0) if xi is positive*<br>
*2.Remaining two weights W0 and W2 will increase the probability of being output be positive(class 1) if xi is positive, Here as W0 is more dominant , it will decide the output class to be 1 or 0 to the maximum extent*<br>
*3.Alternatively, W1 is negative it will increase the probability of being output be positive(class 0) if xi is negative*<br>
*4.Similarly, Remaining two weights W0 and W2 will increase the probability of being output be negative(class 0) if xi is negative, Here as W0 is more dominant , it will decide the output class to be 1 or 0 to the maximum extent*<br>

In [14]:
w = clf_svm.coef_
y_tilda = []
for i in range(len(data)):
    temp = np.dot(w[0],np.array(data.iloc[i,0:3]))
    if(temp<0):
        y_tilda.append(0)
    elif(temp>0):
        y_tilda.append(1)
df_svm = data.copy()
df_svm["y_tilda"]=y_tilda
df_svm[df_svm["y_tilda"]!=df_svm["y"]]
# so here we ended up with wrong output for 95 values with the optimised weight vector
# Also this is due to non-standardization of feature data and we are wrongly predicting
## nearly 50% of the data

,f1,f2,f3,y,y_tilda
4,-768.812047,-7963.932192,1.870536,0.0,1
5,192.093461,-12677.139687,3.456229,0.0,1
8,-31.315888,-15289.241646,3.882981,0.0,1
10,355.555296,-16420.510738,1.968104,0.0,1
11,-371.562551,-2334.218517,2.231601,0.0,1
...,...,...,...,...,...
191,-157.066935,12149.075908,6.944223,1.0,0
192,841.946716,-3756.578733,0.430296,0.0,1
195,119.423142,-2985.720392,0.929967,0.0,1
198,443.199825,-1053.252455,5.467800,0.0,1


**Task 2: Feature importance using Logistic Regression after standardization**

In [15]:
## Standaradising the features data as they are having high varaince and output prediction 
## is going wrong
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [16]:
clf_lr_std = linear_model.SGDClassifier(loss='log',penalty='l2',alpha=0.0001,eta0=0.0001,random_state=15,verbose=2,learning_rate="constant")
clf_lr_std

SGDClassifier(eta0=0.0001, learning_rate='constant', loss='log',
              random_state=15, verbose=2)

In [17]:
clf_lr_std.fit(X,Y)

-- Epoch 1
Norm: 0.01, NNZs: 3, Bias: 0.000001, T: 200, Avg. loss: 0.691431
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 0.02, NNZs: 3, Bias: 0.000002, T: 400, Avg. loss: 0.687922
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 0.03, NNZs: 3, Bias: 0.000002, T: 600, Avg. loss: 0.684449
Total training time: 0.00 seconds.
-- Epoch 4
Norm: 0.03, NNZs: 3, Bias: 0.000003, T: 800, Avg. loss: 0.681011
Total training time: 0.00 seconds.
-- Epoch 5
Norm: 0.04, NNZs: 3, Bias: 0.000003, T: 1000, Avg. loss: 0.677608
Total training time: 0.00 seconds.
-- Epoch 6
Norm: 0.05, NNZs: 3, Bias: 0.000003, T: 1200, Avg. loss: 0.674240
Total training time: 0.00 seconds.
-- Epoch 7
Norm: 0.06, NNZs: 3, Bias: 0.000003, T: 1400, Avg. loss: 0.670905
Total training time: 0.00 seconds.
-- Epoch 8
Norm: 0.07, NNZs: 3, Bias: 0.000003, T: 1600, Avg. loss: 0.667605
Total training time: 0.00 seconds.
-- Epoch 9
Norm: 0.07, NNZs: 3, Bias: 0.000002, T: 1800, Avg. loss: 0.664338
Total training time: 0.00 secon

SGDClassifier(eta0=0.0001, learning_rate='constant', loss='log',
              random_state=15, verbose=2)

In [18]:
clf_lr_std.coef_

array([[ 0.03851242, -0.00553122,  0.88963322]])

**Observations:**<br>
*1. Here we can clearly see that the weights completely changed after standardising the features as because f1 is having more high range of values causing the weight also to make high*<br>
*2. So Ideally W2 is more important in determining the output label, because for a positive xi, it will make more positive and for a negative xi it will tend to more negative class*<br>
*3. W0 is having next highest importance and behaves in the same way as W2*<br>
*4. W1 is having less weight and if xi is negative it will be of very less movement to be positive, in the same way if xi is positive it will slightly tend towards negative class*

In [19]:
w = clf_lr_std.coef_
y_tilda = []
for i in range(len(X)):
    temp = np.dot(w[0],np.array(X[i]))# dot product of weight vector and standardised feats
    if(temp<0):
        y_tilda.append(0)
    elif(temp>0):
        y_tilda.append(1)
# converting "data" data frame into standaradised format to make reading visible
df_lr_std = pd.DataFrame(X,columns=["f1","f2","f3"])
df_lr_std["y"] = Y
df_lr_std["y_tilda"]=y_tilda
print(df_lr_std[df_lr_std["y_tilda"]!=df_lr_std["y"]],"\n",
      df_lr_std[df_lr_std["y_tilda"]!=df_lr_std["y"]].shape)

           f1        f2        f3    y  y_tilda
1   -2.520394 -0.517290 -0.200648  1.0        0
15   0.943057  0.438012  0.065379  0.0        1
28   0.517737 -0.214087 -0.035179  1.0        0
31   0.348143 -0.447289  0.180963  0.0        1
36   0.461469 -0.482392 -0.139486  1.0        0
40   2.300804  0.249974  0.084692  0.0        1
50   1.600541 -0.063459 -0.232022  1.0        0
74  -1.308833  0.147053 -0.035927  1.0        0
95  -0.394592  0.809318  0.177856  0.0        1
96  -0.818095  0.021554 -0.020103  1.0        0
145  1.402883 -0.914822 -0.012546  0.0        1
153 -0.658206  1.640986 -0.142281  1.0        0
159  1.940247  1.368369  0.069814  0.0        1
162  2.168675  1.094113  0.253738  0.0        1
164  1.722800  1.153099 -0.244297  1.0        0
181  1.594267 -0.729767  0.241171  0.0        1
187  1.056864  1.320062  0.031328  0.0        1
189 -1.311197 -0.426307 -0.173383  1.0        0
196 -0.098538  1.133361  0.143667  0.0        1
198  0.889207 -0.226766  0.159612  0.0  

**Observations:**<br>
*1.Now only 20 points are misclassified after standardisation process*<br>
*2.So as mentioned above the more absolute value of weight will cause the impact on the output label either for positive class or negative class depends on the polarity*

**Task 2: Feature importance using SVM(hinge loss) after standardization**

In [20]:
clf_svm_std = linear_model.SGDClassifier(loss='hinge',penalty='l2',alpha=0.0001,eta0=0.0001,random_state=15,verbose=2,learning_rate="constant")
clf_svm_std

SGDClassifier(eta0=0.0001, learning_rate='constant', random_state=15, verbose=2)

In [21]:
clf_svm_std.fit(X,Y)

-- Epoch 1
Norm: 0.02, NNZs: 3, Bias: 0.000000, T: 200, Avg. loss: 0.993111
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 0.03, NNZs: 3, Bias: -0.000000, T: 400, Avg. loss: 0.978934
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 0.05, NNZs: 3, Bias: 0.000000, T: 600, Avg. loss: 0.964757
Total training time: 0.00 seconds.
-- Epoch 4
Norm: 0.07, NNZs: 3, Bias: 0.000000, T: 800, Avg. loss: 0.950580
Total training time: 0.00 seconds.
-- Epoch 5
Norm: 0.08, NNZs: 3, Bias: -0.000000, T: 1000, Avg. loss: 0.936403
Total training time: 0.00 seconds.
-- Epoch 6
Norm: 0.10, NNZs: 3, Bias: 0.000000, T: 1200, Avg. loss: 0.922226
Total training time: 0.00 seconds.
-- Epoch 7
Norm: 0.12, NNZs: 3, Bias: 0.000000, T: 1400, Avg. loss: 0.908049
Total training time: 0.00 seconds.
-- Epoch 8
Norm: 0.13, NNZs: 3, Bias: 0.000000, T: 1600, Avg. loss: 0.893873
Total training time: 0.00 seconds.
-- Epoch 9
Norm: 0.15, NNZs: 3, Bias: 0.000000, T: 1800, Avg. loss: 0.879696
Total training time: 0.00 sec

SGDClassifier(eta0=0.0001, learning_rate='constant', random_state=15, verbose=2)

In [22]:
clf_svm_std.coef_

array([[0.04248904, 0.02585179, 1.07272982]])

**Observations:**<br>
*1. Here we can clearly see that the weights completely changed after standardising the features as because f1 is having more high range of values(in the input data csv -high variance) causing the weight also to make high*<br>
*2. So Ideally W2 is more important in determining the output label, because for a positive xi, it will make more positive and for a negative xi it will tend to more negative class*<br>
*3. W0 is having next highest importance and behaves in the same way as W2*<br>
*4. W1 is having less weight and if xi is negative it will be of very less movement to be negative, in the same way if xi is positive it will slightly tend towards positive class*

In [23]:
w = clf_svm_std.coef_
y_tilda = []
for i in range(len(X)):
    temp = np.dot(w[0],np.array(X[i])) # dot product of weight vector and standardised feats
    if(temp<0):
        y_tilda.append(0)
    elif(temp>0):
        y_tilda.append(1)
# converting "data" data frame into standaradised format to make reading visible
df_svm_std = pd.DataFrame(X,columns=["f1","f2","f3"])
df_svm_std["y"] = Y
df_svm_std["y_tilda"]=y_tilda
print(df_svm_std[df_svm_std["y_tilda"]!=df_svm_std["y"]],"\n",
      df_svm_std[df_svm_std["y_tilda"]!=df_svm_std["y"]].shape)

           f1        f2        f3    y  y_tilda
1   -2.520394 -0.517290 -0.200648  1.0        0
15   0.943057  0.438012  0.065379  0.0        1
28   0.517737 -0.214087 -0.035179  1.0        0
31   0.348143 -0.447289  0.180963  0.0        1
36   0.461469 -0.482392 -0.139486  1.0        0
40   2.300804  0.249974  0.084692  0.0        1
50   1.600541 -0.063459 -0.232022  1.0        0
74  -1.308833  0.147053 -0.035927  1.0        0
95  -0.394592  0.809318  0.177856  0.0        1
96  -0.818095  0.021554 -0.020103  1.0        0
145  1.402883 -0.914822 -0.012546  0.0        1
153 -0.658206  1.640986 -0.142281  1.0        0
159  1.940247  1.368369  0.069814  0.0        1
162  2.168675  1.094113  0.253738  0.0        1
164  1.722800  1.153099 -0.244297  1.0        0
181  1.594267 -0.729767  0.241171  0.0        1
187  1.056864  1.320062  0.031328  0.0        1
189 -1.311197 -0.426307 -0.173383  1.0        0
196 -0.098538  1.133361  0.143667  0.0        1
198  0.889207 -0.226766  0.159612  0.0  

**Observations:**<br>
*1.Now only 20 points are misclassified after standardisation process and 180 points are correctly classified now*<br>
*2.So as mentioned above the more absolute value of weight will cause the impact on the output label either for positive class or negative class depends on the polarity*<br>
*3. As Linear SVM is almost equivalent to Logistic regression, we are getting mostly the same outputs even thought weight vector have subtle difference in magnitude*<br>
*4. The most important thing is if standardisation is not done, we may end up in treating a unimportant feature as an important feature due to its high variance factor as seen before*